# DTN Transfers over NVMEoF

This notebook demonstrates a copy from one DTN to another DTN.

In [ ]:
# initialize client and run a connectivity check to the orchestrator container
from dtn_orchestrator.client import DTNOrchestratorClient
client = DTNOrchestratorClient()
client.check()

In [ ]:
# Optional - get documentation for using the orchestrator client
help(client)

In [ ]:
# register DTNs with their management address - this will reuse existing objects if they're already registered
# sender DTN
dtn1 = client.register_dtn('<ip1>:5000')
# receiver DTN
dtn2 = client.register_dtn('<ip2>:5000')

In [ ]:
# Run latency between DTNs to test connectivity
print(dtn1.ping(dtn2.id))
print(dtn2.ping(dtn1.id))

In [ ]:
# Set up a connection between DTNs - this provides an object that can be used to start the transfer
conn = client.setup_connection(dtn1, dtn2, tool="nuttcp")

In [ ]:
# Optional - get documentation for using the connection object
help(conn)

In [ ]:
# Get a list of files on each DTN
print([f['name'] for f in dtn1.files() if f['type'] == 'file'])
print([f['name'] for f in dtn2.files() if f['type'] == 'file'])

In [ ]:
# Copy a directory on DTN1 to an empty directory on DTN2 
# - "limit" can also be passed to limit the number of files copied
result = conn.copy('src', 'dst')
print(result)

In [ ]:
# Periodically get the status of the transfer, and stop when the transfer is complete
import time
finished = False
while not finished:
    status = conn.status()
    print(status)
    finished = status.get('Unfinished') == 0
    time.sleep(30)
# now finish the connection so we can get transfer statistics
conn.finish()

In [ ]:
# Get transfer statistics - start time, data transferred, number of files copied, etc.
conn.get()